In [1]:
import os
import numpy as np
from PIL import Image, ImageDraw, ImageFont
import cv2
import random
import string
from sklearn.model_selection import train_test_split

def collect_font_files(fonts_dir):
    '''
    Сбор файлов шрифтов из указанной директории.
    
    Args:
    fonts_dir (str): Путь к директории с файлами шрифтов.
    
    Returns:
    list: Список путей к файлам шрифтов (.ttf и .otf).
    '''
    font_files = []
    for root, dirs, files in os.walk(fonts_dir):
        for file in files:
            if file.endswith(".ttf") or file.endswith(".otf"):
                font_files.append(os.path.join(root, file))
    return font_files

def generate_images(fonts, output_dir="dataset", num_images=100, img_size=(2048, 256)):
    '''
    Генерация изображений строк с использованием различных шрифтов и сохранение их в файлы.
    
    Args:
    fonts (list): Список файлов шрифтов.
    output_dir (str): Директория для сохранения изображений.
    num_images (int): Количество изображений для генерации на каждый шрифт.
    img_size (tuple): Размер изображения (ширина, высота).
    
    Returns:
    tuple: массив numpy изображений символов и массив меток соответствующих шрифтов.
    '''
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    font_size = 200
    char_images = []
    char_labels = []
    for font_file in fonts:
        font_name = os.path.splitext(os.path.basename(font_file))[0]
        font_output_dir = os.path.join(output_dir, font_name)
        if not os.path.exists(font_output_dir):
            os.makedirs(font_output_dir)

        for i in range(num_images):
            text = random_string(random.randint(1, 15))
            image = Image.new("L", img_size, color=255)
            draw = ImageDraw.Draw(image)
            font = ImageFont.truetype(font_file, size=font_size)
            draw.text((10, 10), text, font=font, fill=0)
            image_path = os.path.join(font_output_dir, f"{font_name}_{i}.png")
            image.save(image_path)
            image_array = np.array(image)
            
            contours = find_contours(image)
            symbols = extract_symbols(image, contours)
            for symbol in symbols:
                char_images.append(symbol)
                char_labels.append(font_name)

    return np.array(char_images), np.array(char_labels)

def random_string(length):
    '''
    Генерация случайной строки из латинских букв и цифр.
    
    Args:
    length (int): Длина генерируемой строки.
    
    Returns:
    str: Сгенерированная строка.
    '''
    letters = string.ascii_letters + string.digits
    return ''.join(random.choice(letters) for i in range(length))

def find_contours(char_image):
    '''
    Поиск контуров в изображении для выделения отдельных символов.
    
    Args:
    char_image (Image): Изображение, из которого необходимо извлечь контуры.
    
    Returns:
    list: Список прямоугольников, описывающих найденные контуры.
    '''
    im_array = np.array(char_image)
    if len(im_array.shape) == 3:
        im_gray = cv2.cvtColor(im_array, cv2.COLOR_RGB2GRAY)
    else:
        im_gray = im_array

    im_th = cv2.adaptiveThreshold(im_gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, 
                                  cv2.THRESH_BINARY_INV, 11, 2)
    ctrs, _ = cv2.findContours(im_th, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    contours = [cv2.boundingRect(ctr) for ctr in ctrs]
    return contours

def extract_symbols(char_image, contours):
    '''
    Выделение символов из строки по найденным контурам.
    
    Args:
    char_image (Image): Изображение строки.
    contours (list): Список контуров символов.
    
    Returns:
    list: Список изображений выделенных символов.
    '''
    symbols = []
    for (x, y, w, h) in contours:
        symbol = char_image.crop((x, y, x + w, y + h))
        symbol = symbol.resize((28, 28))
        symbols.append(np.array(symbol))
    return symbols


# Путь к директории с шрифтами
fonts_dir = "fonts"

# Сбор файлов шрифтов
fonts = collect_font_files(fonts_dir)

# Генерация изображений и меток
images, labels = generate_images(fonts)

# Разделение данных на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)

# Сохранение обучающую и тестовую выборки
np.savez("fonts_dataset.npz", X_train=X_train, X_test=X_test, y_train=y_train, y_test=y_test, labels=labels)